In [2]:
#!pip install langchain openai chromadb unstructured tiktoken

In [7]:
#!pip install unstructured[pdf] pdf2image
#!pip install -U langchain-community

In [37]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
# 📚 Étape 1 – Chargement du fichier PDF (ou TXT)


from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader


# Remplace par ton chemin de fichier
#loader = UnstructuredPDFLoader("madame-bovary_Gustave-flaubert.pdf")
loader = PyPDFLoader("documents/madame-bovary_Gustave-flaubert.pdf")
docs = loader.load()

print(f"{len(docs)} pages chargées")

420 pages chargées


In [1]:
#docs[0:4]

In [2]:
#print(docs[0].page_content[:])

In [3]:
#print(docs[419].page_content)

## Text Pre-Processing

In [4]:
from langchain.document_loaders import PyPDFLoader

# Charger le document
loader = PyPDFLoader("documents/madame-bovary_Gustave-flaubert.pdf")
docs = loader.load()

# Fusionner tout le texte dans un seul bloc pour faciliter la recherche
full_text = "\n".join([doc.page_content for doc in docs])

# Exemple simple pour diviser par parties en s'appuyant sur les titres visibles dans la table des matières
import re

# On cherche les grands titres de parties (PREMIÈRE PARTIE, DEUXIÈME PARTIE, TROISIÈME PARTIE, etc.)
chapitres = re.split(r"\n+(PREMIÈRE PARTIE|DEUXIÈME PARTIE|TROISIÈME PARTIE)\n+", full_text)

# Maintenant, on crée un dictionnaire pour stocker tout ça
chapitres_dict = {}
for i in range(1, len(chapitres), 2):
    partie = chapitres[i]
    contenu = chapitres[i+1]
    chapitres_dict[partie] = contenu.strip()

# Pour afficher les titres et un extrait de chaque partie
for titre, texte in chapitres_dict.items():
    print(f"{titre} : {texte[:200]}...\n")


In [6]:
#full_text
#chapitres

In [8]:
from langchain.document_loaders import PyPDFLoader

# Charger le PDF
loader = PyPDFLoader("documents/madame-bovary_Gustave-flaubert.pdf")
docs = loader.load()

# Mapping des pages par index (attention : index 0 = page 1 du PDF)
page_map = {doc.metadata["page"] + 1: doc.page_content for doc in docs}

# Table des matières extraite manuellement
table_matiere = {
    "PREMIÈRE PARTIE": {
        "I": 5, "II": 15, "III": 24, "IV": 30, "V": 35, "VI": 39, "VII": 44, "VIII": 51, "IX": 62
    },
    "DEUXIÈME PARTIE": {
        "I": 75, "II": 85, "III": 93, "IV": 105, "V": 109, "VI": 120, "VII": 137,
        "VIII": 148, "IX": 175, "X": 189, "XI": 200, "XII": 214, "XIII": 232, "XIV": 244, "XV": 256
    },
    "TROISIÈME PARTIE": {
        "I": 267, "II": 286, "III": 299, "IV": 302, "V": 306, "VI": 327, "VII": 349,
        "VIII": 368, "IX": 391, "X": 402, "XI": 409
    }
}

# Fonction pour extraire les chapitres
def extraire_chapitres(pages_dict, toc):
    result = {}
    for partie, chapitres in toc.items():
        partie_dict = {}
        titres = list(chapitres.keys())
        pages = list(chapitres.values())
        
        for i, titre in enumerate(titres):
            start_page = pages[i]
            end_page = pages[i+1] if i + 1 < len(pages) else None

            # Extraire le texte des pages concernées
            contenu = []
            for page_num in range(start_page, end_page if end_page else 420):
                if page_num in pages_dict:
                    contenu.append(pages_dict[page_num])
            partie_dict[titre] = "\n".join(contenu).strip()
        
        result[partie] = partie_dict
    return result

# Exécution
chapitres_structures = extraire_chapitres(page_map, table_matiere)
type(chapitres_structures)

dict

In [10]:
len(chapitres_structures)

3

In [24]:
print(f"Part 1 : {len(chapitres_structures["PREMIÈRE PARTIE"])} Chapitres" )
print(f"Part 2 : {len(chapitres_structures["DEUXIÈME PARTIE"])} Chapitres" )
print(f"Part 3 : {len(chapitres_structures["TROISIÈME PARTIE"])} Chapitres" )

Part 1 : 9 Chapitres
Part 2 : 15 Chapitres
Part 3 : 11 Chapitres


In [56]:
part1_chap1 = chapitres_structures["PREMIÈRE PARTIE"]["I"]
print(f"Type Chapitre 1 : {type(part1_chap1)} ")
print(f"Taille Chapitre 1: {len(part1_chap1)} ")

# Exemple : Afficher un extrait du chapitre I de la PREMIÈRE PARTIE
print(chapitres_structures["PREMIÈRE PARTIE"]["I"][:100])

Type Chapitre 1 : <class 'str'> 
Taille Chapitre 1: 19248 
– 5 – 
PREMIÈRE PARTIE 
 
I 
 
Nous étions à l’Étude, quand le  Proviseur entra, suivi d’un 
nouveau


In [57]:
part1_chap9 = chapitres_structures["PREMIÈRE PARTIE"]["IX"]
print(f"Type Chapitre 9 : {type(part1_chap9)} ")
print(f"Taille Chapitre 9: {len(part1_chap9)} ")

# Exemple : Afficher un extrait du chapitre IX de la PREMIÈRE PARTIE
print(chapitres_structures["PREMIÈRE PARTIE"]["IX"][:1000])

Type Chapitre 9 : <class 'str'> 
Taille Chapitre 9: 594129 
– 62 – 
frottement de la richesse, il s’était placé dessus quelque chose qui 
ne s’effacerait pas. 
 
Ce fut donc une occupation pour Emma que le souvenir de ce 
bal. Toutes les fois que revenait le mercredi, elle se disait en 
s’éveillant : « Ah ! il y a huit jours… il y a quinze jours…, il y a 
trois semaines, j’y étais ! » Et peu à peu, les physionomies se 
confondirent dans sa mémoire, elle oublia l’air des contredanses, 
elle ne vit plus si nettement les livrées et les appartements ; quel-
ques détails s’en allèrent ; mais le regret lui resta. 
 
IX 
 
Souvent, lorsque Charles était sorti, elle allait prendre dans 
l’armoire, entre les plis du linge où elle l’avait laissé, le porte-
cigares en soie verte. 
 
Elle le regardait, l’ouvrait, et même elle flairait l’odeur de sa 
doublure, mêlée de verveine et de tabac. À qui appartenait-il ?… 
Au Vicomte. C’était peut-être un cadeau de sa maîtresse. On avait 
brodé cela sur qu

In [60]:
print(chapitres_structures["PREMIÈRE PARTIE"]["IX"][:])

– 62 – 
frottement de la richesse, il s’était placé dessus quelque chose qui 
ne s’effacerait pas. 
 
Ce fut donc une occupation pour Emma que le souvenir de ce 
bal. Toutes les fois que revenait le mercredi, elle se disait en 
s’éveillant : « Ah ! il y a huit jours… il y a quinze jours…, il y a 
trois semaines, j’y étais ! » Et peu à peu, les physionomies se 
confondirent dans sa mémoire, elle oublia l’air des contredanses, 
elle ne vit plus si nettement les livrées et les appartements ; quel-
ques détails s’en allèrent ; mais le regret lui resta. 
 
IX 
 
Souvent, lorsque Charles était sorti, elle allait prendre dans 
l’armoire, entre les plis du linge où elle l’avait laissé, le porte-
cigares en soie verte. 
 
Elle le regardait, l’ouvrait, et même elle flairait l’odeur de sa 
doublure, mêlée de verveine et de tabac. À qui appartenait-il ?… 
Au Vicomte. C’était peut-être un cadeau de sa maîtresse. On avait 
brodé cela sur quelque métier de palissandre, meuble mignon que 
l’on cachait

In [62]:
with open('part1_chap9.txt', 'r', encoding='utf-8') as fichier:
    part1_chap9 = fichier.read()

In [65]:
#print(part1_chap9)

In [38]:
from langchain.prompts import PromptTemplate

character_extraction_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un assistant littéraire expert en analyse de romans.

Voici un extrait de livre :

-------------------
{context}
-------------------

Ta tâche est d'identifier **tous les personnages mentionnés dans ce passage**, même brièvement.
Pour chaque personnage :
- Donne **le nom complet** (ou partiel s'il n'y a que cela)
- Indique **un court résumé de son rôle ou importance** dans le récit (si possible)
- Mentionne les relations ou interactions importantes s’il y en a

Formate ta réponse comme ceci :

1. Nom : ...
   Rôle : ...
   Liens : ...

2. Nom : ...
   ...

Ne fais **aucune supposition** en dehors du texte donné. Si aucun personnage n’est mentionné, dis "Aucun personnage identifié."
"""
)


In [41]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
character_chain = LLMChain(llm=llm, prompt=character_extraction_prompt)


# Exemple avec un chunk du livre
part1_chap1 = chapitres_structures["PREMIÈRE PARTIE"]["I"]
context = part1_chap1

result = character_chain.run(context=context)
print(result)

1. Nom : Charles Bovary (aussi appelé "le nouveau", "Charbovari")
   Rôle : Personnage principal du passage, nouvel élève entrant en cinquième, dont on suit l’arrivée au collège, l’enfance, l’éducation et le début de sa vie d’adulte.
   Liens : Fils de M. Charles-Denis-Bartholomé Bovary et de Madame Bovary (sa mère), futur médecin, élève du collège de Rouen.

2. Nom : M. Charles-Denis-Bartholomé Bovary
   Rôle : Père de Charles Bovary, ancien aide-chirurgien-major, homme dépensier et peu impliqué dans l’éducation de son fils.
   Liens : Mari de Madame Bovary (la mère), père de Charles Bovary.

3. Nom : Madame Bovary (la mère de Charles, prénom non donné)
   Rôle : Mère de Charles Bovary, femme dévouée à son fils, très impliquée dans son éducation et sa réussite.
   Liens : Femme de M. Charles-Denis-Bartholomé Bovary, mère de Charles Bovary.

4. Nom : Monsieur Roger
   Rôle : Maître d’études à l’école où Charles arrive.
   Liens : Reçoit Charles Bovary sur recommandation du Proviseur.



In [23]:
context = chunks[20:30]
context

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
character_chain = LLMChain(llm=llm, prompt=character_extraction_prompt)

# Exemple avec un chunk du livre
result = character_chain.run(context=context)
print(result)

1. Nom : Charles Bovary (souvent appelé simplement "Charles" ou "M. Bovary")
   Rôle : Médecin de campagne, personnage principal du passage, appelé à la ferme des Bertaux pour soigner une jambe cassée.
   Liens : Mari de "Madame Bovary jeune", médecin du père Rouault, rencontre et interactions avec Emma Rouault.

2. Nom : Madame Bovary jeune
   Rôle : Épouse de Charles Bovary, inquiète pour son mari lors de son départ nocturne.
   Liens : Femme de Charles Bovary.

3. Nom : Nastasie
   Rôle : Bonne (servante) présente lors de la lecture de la lettre, tient la lumière.
   Liens : Employée chez les Bovary.

4. Nom : Valet d’écurie
   Rôle : Employé chargé de partir en avant pour préparer le chemin de Charles vers la ferme des Bertaux.
   Liens : Employé chez les Bovary.

5. Nom : Gamin (jeune garçon)
   Rôle : Enfant envoyé à la rencontre de Charles pour lui montrer le chemin de la ferme et ouvrir les clôtures.
   Liens : Guide Charles jusqu’aux Bertaux.

6. Nom : M. Rouault
   Rôle : Cul

In [16]:
from langchain.prompts import PromptTemplate

summary_template = PromptTemplate(
    input_variables=["chapter_title"],
    template="""
Tu es un expert en littérature. Tu lis un extrait de livre et ton objectif est de **résumer le chapitre** intitulé : "{chapter_title}".

Consignes :
- Résume en **français clair**.
- Détaille les **idées principales**, les **événements clés**, et les **personnages impliqués**.
- Sois concis mais informatif (environ 150-300 mots).

Voici le contenu extrait du chapitre :
{context}

Résumé :
""",
)


In [29]:
context = chunks[0:20]

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

chapter_summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 500 mots

Résumé :
""",
)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)

# Exemple avec un chunk du livre
result = chapter_summary.run(context=context)
print(result)

**Titre : Les Origines et la Jeunesse de Charles Bovary**

**Résumé du premier chapitre de "Madame Bovary" de Gustave Flaubert**

Le premier chapitre de "Madame Bovary" s’ouvre sur l’arrivée d’un nouvel élève dans une classe de collège : Charles Bovary. La scène, racontée à la première personne du pluriel, plonge le lecteur dans l’ambiance scolaire, marquée par la curiosité et la moquerie des élèves envers ce garçon maladroit, vêtu de façon provinciale et manifestement peu à l’aise dans ce nouvel environnement. L’épisode de la casquette, objet ridicule et source de railleries, symbolise d’emblée la gaucherie et la timidité de Charles, qui peine à s’intégrer.

Le professeur, après avoir calmé la classe, tente d’obtenir le nom du nouveau, mais Charles, intimidé, le prononce de façon incompréhensible, déclenchant un chahut général. Finalement, son nom, Charles Bovary, est compris et il est installé sur le banc des paresseux, déjà marqué par la différence et l’exclusion.

Le récit s’attard

In [28]:
context = chunks[0:20]

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

chapter_summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 1000-1500 mots

Résumé :
""",
)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)

# Exemple avec un chunk du livre
result = chapter_summary.run(context=context)
print(result)

**Titre du chapitre :**  
**Les Origines et la Formation de Charles Bovary**

---

**Résumé du chapitre I de la Première Partie de "Madame Bovary"**

Ce premier chapitre de "Madame Bovary" de Gustave Flaubert s’attache à présenter le personnage de Charles Bovary, futur mari d’Emma, en retraçant son enfance, son éducation, ses origines familiales et ses premiers pas dans la vie adulte. À travers une narration précise et détaillée, Flaubert dresse le portrait d’un homme ordinaire, sans éclat, dont la médiocrité et la passivité vont marquer l’ensemble du roman.

---

### 1. **L’arrivée de Charles au collège : la scène inaugurale**

Le récit s’ouvre sur une scène de classe, vue à la première personne du pluriel, qui plonge le lecteur dans l’ambiance d’un collège de province. Un nouveau venu, Charles Bovary, fait son entrée, accompagné du proviseur et d’un garçon de classe. Son apparence rustique, sa maladresse et sa timidité le distinguent immédiatement des autres élèves. Son habit mal aju

In [27]:
context = chunks[20:30]

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI



chapter_summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 1000-1500 mots

Résumé :
""",
)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)

# Exemple avec un chunk du livre
result = chapter_summary.run(context=context)
print(result)

**Titre du chapitre :**  
**La rencontre de Charles Bovary et d’Emma Rouault aux Bertaux**

---

**Résumé détaillé :**

Ce chapitre de *Madame Bovary* de Gustave Flaubert marque un tournant décisif dans la vie de Charles Bovary, jeune officier de santé récemment marié, et introduit pour la première fois de façon significative le personnage d’Emma Rouault, future Madame Bovary. Il s’agit d’un chapitre charnière, où se nouent les premiers liens entre Charles et Emma, dans le cadre rural de la ferme des Bertaux.

---

### 1. **L’appel aux Bertaux : le départ de Charles**

L’action débute alors que Charles Bovary, installé à Tostes, reçoit en pleine nuit une lettre urgente. Celle-ci, cachetée de cire bleue, le supplie de se rendre sans délai à la ferme des Bertaux, où le propriétaire, M. Rouault, s’est cassé la jambe. La distance est importante (six lieues), la nuit noire, et la jeune épouse de Charles, inquiète, prend des précautions pour la sécurité de son mari : un valet d’écurie part e

In [26]:
#context = chunks[20:30]
context = part1_chap3

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI



chapter_summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 1000-1500 mots

Résumé :
""",
)

llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)

# Exemple avec un chunk du livre
result = chapter_summary.run(context=context)
print(result)

/tmp/ipykernel_841/2480877406.py:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)


**Titre :**  
**La demande en mariage de Charles Bovary et les préparatifs de son union avec Emma Rouault**

**Résumé :**

Ce chapitre marque un tournant décisif dans la vie de Charles Bovary, jeune médecin de campagne récemment veuf, et d’Emma Rouault, la fille du fermier des Bertaux. Il s’agit d’un moment charnière où le deuil de Charles cède progressivement la place à l’espoir d’une nouvelle vie, tandis qu’Emma, encore marquée par l’ennui de la campagne et la nostalgie de son passé, s’apprête à entrer dans le monde du mariage.

**1. La visite du père Rouault et la consolation du deuil**

Le chapitre s’ouvre sur la visite du père Rouault à Charles Bovary. Il vient régler la dette contractée pour la guérison de sa jambe, apportant soixante-quinze francs et une dinde. Mais au-delà de ce paiement, il manifeste une réelle empathie envers Charles, qui vient de perdre sa femme. Le père Rouault partage son expérience personnelle du deuil, évoquant la perte de sa propre épouse et la douleur 

In [29]:
from langchain.prompts import PromptTemplate

character_extraction_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un assistant littéraire expert en analyse de romans.

Voici un extrait de livre :

-------------------
{context}
-------------------

Ta tâche est d'identifier **tous les personnages mentionnés dans ce passage**, même brièvement.
Pour chaque personnage :
- Donne **le nom complet** (ou partiel s'il n'y a que cela)
- Indique **un court résumé de son rôle ou importance** dans le récit (si possible)
- Mentionne les relations ou interactions importantes s’il y en a

Formate ta réponse comme ceci :

1. Nom : ...
   Rôle : ...
   Liens : ...

2. Nom : ...
   ...

Ne fais **aucune supposition** en dehors du texte donné. Si aucun personnage n’est mentionné, dis "Aucun personnage identifié."
"""
)



from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
character_chain = LLMChain(llm=llm, prompt=character_extraction_prompt)


context = part1_chap3
# Exemple avec un chunk du livre
result = character_chain.run(context=context)
print(result)

1. Nom : Père Rouault  
   Rôle : Père d’Emma, fermier, client de Charles (il lui apporte le paiement pour la jambe remise).  
   Liens : Père d’Emma Rouault, relation amicale et de confiance avec Charles Bovary, souhaite marier sa fille à Charles.

2. Nom : Charles (Monsieur Bovary, Charles Bovary)  
   Rôle : Médecin, veuf, personnage central du passage, reçoit la visite du père Rouault, finit par demander Emma en mariage.  
   Liens : Médecin du père Rouault, futur mari d’Emma Rouault, veuf (sa femme est décédée).

3. Nom : Emma (Mademoiselle Rouault)  
   Rôle : Fille du père Rouault, future épouse de Charles, objet de l’attention et de l’affection de Charles.  
   Liens : Fille du père Rouault, future épouse de Charles Bovary.

4. Nom : La défunte (femme décédée du père Rouault)  
   Rôle : Ancienne épouse du père Rouault, mentionnée dans le récit du père Rouault sur son propre deuil.  
   Liens : Femme décédée du père Rouault.

5. Nom : Femme décédée de Charles (sa première femme

In [30]:
#context = chunks[20:30]
context = part1_chap3

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI



chapter_summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 300-400 mots

Résumé :
""",
)

llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)

# Exemple avec un chunk du livre
result = chapter_summary.run(context=context)
print(result)

**Titre : La demande en mariage de Charles Bovary et ses fiançailles avec Emma Rouault**

Dans ce chapitre, Charles Bovary, récemment veuf, reçoit la visite du père Rouault, venu lui régler ses honoraires pour l’avoir soigné. Le père Rouault, compatissant, partage avec Charles son expérience du deuil et l’encourage à ne pas se laisser abattre, l’invitant à venir plus souvent aux Bertaux, la ferme familiale. Il mentionne également qu’Emma, sa fille, pense à Charles, suggérant ainsi une possible proximité entre eux.

Charles suit ce conseil et retourne aux Bertaux, où il retrouve Emma. Leur relation se développe dans une atmosphère de simplicité rurale et de discrétion. Charles, d’abord accablé par la solitude, commence à apprécier sa nouvelle indépendance et se sent peu à peu attiré par Emma. Lors de ses visites, il partage avec elle des moments de complicité, marqués par des échanges sur leur passé respectif et des gestes de familiarité. Emma, de son côté, se montre tantôt rêveuse, tan

In [42]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI


chapter_summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 1000-1500 mots

Résumé :
""",
)

llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)


# Exemple avec un chapitre du livre
part1_chap4 = chapitres_structures["PREMIÈRE PARTIE"]["IV"]
context = part1_chap4

# Exemple avec un chunk du livre
result = chapter_summary.run(context=context)
print(result)

**Titre : Le Mariage de Charles et Emma : Chronique d’une Noce Campagnarde**

**Résumé**

Ce chapitre relate en détail le mariage de Charles Bovary et d’Emma Rouault, événement central du début du roman, et en dresse un tableau vivant, foisonnant de détails sur la société rurale normande du XIXe siècle. À travers la description minutieuse des préparatifs, de la cérémonie et des festivités, Gustave Flaubert met en scène les principaux personnages et pose les bases des dynamiques familiales et sociales qui influenceront la suite du récit.

**Arrivée des invités et atmosphère de la noce**

Dès l’aube, les invités affluent aux Bertaux, la ferme du père Rouault, dans une grande diversité de véhicules : carrioles, chars à bancs, cabriolets, tapissières, charrettes. Certains viennent de loin, jusqu’à dix lieues, témoignant de l’importance de l’événement pour les deux familles et leur entourage. Tous les parents, amis, connaissances, même ceux avec qui l’on était brouillé ou perdus de vue, ont

In [43]:
from langchain.prompts import PromptTemplate

character_extraction_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un assistant littéraire expert en analyse de romans.

Voici un extrait de livre :

-------------------
{context}
-------------------

Ta tâche est d'identifier **tous les personnages mentionnés dans ce passage**, même brièvement.
Pour chaque personnage :
- Donne **le nom complet** (ou partiel s'il n'y a que cela)
- Indique **un court résumé de son rôle ou importance** dans le récit (si possible)
- Mentionne les relations ou interactions importantes s’il y en a

Formate ta réponse comme ceci :

1. Nom : ...
   Rôle : ...
   Liens : ...

2. Nom : ...
   ...

Ne fais **aucune supposition** en dehors du texte donné. Si aucun personnage n’est mentionné, dis "Aucun personnage identifié."
"""
)



from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
character_chain = LLMChain(llm=llm, prompt=character_extraction_prompt)


context = part1_chap4
# Exemple avec un chunk du livre
result = character_chain.run(context=context)
print(result)

1. Nom : Emma (la mariée)
   Rôle : Personnage central de la scène, elle est la mariée dont la robe traîne et qui fait attention à ne pas la salir. 
   Liens : Fille du père Rouault, épouse de Charles, belle-fille de Madame Bovary mère et de M. Bovary père.

2. Nom : Charles (le marié)
   Rôle : Personnage central de la scène, il est le marié, médecin, décrit comme peu facétieux lors de la noce.
   Liens : Fils de Madame Bovary mère et de M. Bovary père, époux d’Emma, gendre du père Rouault.

3. Nom : Père Rouault
   Rôle : Père de la mariée, il accompagne sa fille à la mairie, la protège des plaisanteries, et la reconduit après la noce.
   Liens : Père d’Emma, beau-père de Charles.

4. Nom : Madame Bovary mère
   Rôle : Mère du marié, elle est présente à la noce, silencieuse et peu consultée.
   Liens : Mère de Charles, belle-mère d’Emma, épouse de M. Bovary père.

5. Nom : M. Bovary père
   Rôle : Père du marié, il se distingue par son attitude méprisante envers les autres invités et

In [44]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI



chapter_summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 300-400 mots

Résumé :
""",
)

llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)

# Exemple avec un chunk du livre
context = part1_chap4


result = chapter_summary.run(context=context)
print(result)

**Titre : Le mariage de Charles et Emma : une noce villageoise**

Résumé :

Ce chapitre décrit en détail le mariage de Charles Bovary et d’Emma Rouault, événement central réunissant les familles et la communauté villageoise. Les invités affluent dès le matin, arrivant de divers villages voisins dans des véhicules variés, témoignant de la diversité sociale et de l’importance de la cérémonie. Les familles ont invité un large cercle de parents, réconcilié d’anciens amis et convié des connaissances lointaines, soulignant le caractère exceptionnel de la fête.

La description des arrivées met en avant la diversité des tenues, des attitudes et des origines sociales des convives. Les hommes, vêtus de leurs plus beaux habits ou de blouses de cérémonie, et les femmes, parées de robes à la mode urbaine, affichent une certaine solennité mêlée de maladresse. Les enfants, souvent gênés par leurs vêtements neufs, et les jeunes filles, intimidées, complètent ce tableau vivant et pittoresque.

Après la

In [66]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI


chapter_summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 1000-1500 mots

Résumé :
""",
)

llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)


# Exemple avec un chapitre du livre
with open('part1_chap9.txt', 'r', encoding='utf-8') as fichier:
    part1_chap9 = fichier.read()

context = part1_chap9

# Exemple avec un chunk du livre
result = chapter_summary.run(context=context)
print(result)

**Titre : L’ennui d’Emma à Tostes : Désillusions, Rêves et Déclin**

**Résumé du chapitre IX de Madame Bovary**

Ce chapitre s’attache à décrire la vie quotidienne d’Emma Bovary à Tostes, marquée par l’ennui, la désillusion et la montée d’un profond mal-être. Il s’agit d’un moment charnière du roman, où l’on voit s’installer durablement la crise existentielle d’Emma, son insatisfaction conjugale et sociale, et la naissance de ses fantasmes d’une vie plus brillante et passionnée.

---

**1. Le porte-cigares du Vicomte : symbole d’un passé idéalisé**

Le chapitre s’ouvre sur une scène révélatrice : Emma, souvent seule lorsque Charles est absent, va chercher dans l’armoire un porte-cigares en soie verte, souvenir du bal à la Vaubyessard et du Vicomte qui l’avait oublié. Cet objet, qu’elle caresse, hume et contemple, devient le symbole de ses rêves d’élégance, de raffinement et de passion. Elle imagine la vie du Vicomte, ses amours, la femme qui aurait brodé ce porte-cigares, et projette s

In [67]:
from langchain.prompts import PromptTemplate

character_extraction_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un assistant littéraire expert en analyse de romans.

Voici un extrait de livre :

-------------------
{context}
-------------------

Ta tâche est d'identifier **tous les personnages mentionnés dans ce passage**, même brièvement.
Pour chaque personnage :
- Donne **le nom complet** (ou partiel s'il n'y a que cela)
- Indique **un court résumé de son rôle ou importance** dans le récit (si possible)
- Mentionne les relations ou interactions importantes s’il y en a

Formate ta réponse comme ceci :

1. Nom : ...
   Rôle : ...
   Liens : ...

2. Nom : ...
   ...

Ne fais **aucune supposition** en dehors du texte donné. Si aucun personnage n’est mentionné, dis "Aucun personnage identifié."
"""
)



from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
character_chain = LLMChain(llm=llm, prompt=character_extraction_prompt)


context = part1_chap9
# Exemple avec un chunk du livre
result = character_chain.run(context=context)
print(result)

1. Nom : Charles (Bovary)
   Rôle : Mari d’Emma, médecin de campagne, personnage central du passage.
   Liens : Mari d’Emma, gendre du père Rouault, fils de Madame Bovary mère.

2. Nom : Emma (Bovary)
   Rôle : Héroïne du roman, épouse de Charles, personnage principal du passage.
   Liens : Femme de Charles, fille du père Rouault, belle-fille de Madame Bovary mère, maîtresse de maison à Tostes.

3. Nom : Le Vicomte
   Rôle : Ancien visiteur à la Vaubyessard, propriétaire du porte-cigares qui fascine Emma.
   Liens : Aucun lien direct dans ce passage, mais objet de rêverie pour Emma.

4. Nom : Félicité
   Rôle : Nouvelle bonne d’Emma, jeune fille de quatorze ans, orpheline.
   Liens : Domestique d’Emma, remplace Nastasie.

5. Nom : Nastasie
   Rôle : Ancienne bonne d’Emma, quitte le service en pleurant.
   Liens : Ancienne domestique d’Emma.

6. Nom : Madame Bovary mère
   Rôle : Mère de Charles, belle-mère d’Emma, vient passer une partie du carême à Tostes.
   Liens : Mère de Charles, 

In [68]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI



chapter_summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 300-400 mots

Résumé :
""",
)

llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)

# Exemple avec un chunk du livre
context = part1_chap9


result = chapter_summary.run(context=context)
print(result)

**Titre : L’ennui d’Emma à Tostes et la naissance du désenchantement**

Dans ce chapitre, Flaubert décrit la vie quotidienne d’Emma Bovary à Tostes, marquée par l’ennui, la frustration et la montée d’un profond désenchantement. Emma, insatisfaite de sa vie de province et de son mariage avec Charles, s’évade dans la rêverie et la lecture. Elle conserve précieusement un porte-cigares ayant appartenu au vicomte rencontré lors d’un bal, symbole d’un monde raffiné et inaccessible qui nourrit ses fantasmes. Paris, qu’elle imagine à travers des plans et des journaux féminins, devient pour elle le lieu de toutes les passions et de tous les possibles, en contraste avec la médiocrité de son existence.

Emma tente d’introduire dans son quotidien des touches de luxe et d’élégance, imitant les modes et les habitudes de la haute société, mais ces efforts ne font qu’accentuer son sentiment d’isolement. Elle s’ennuie profondément, délaisse la musique, la couture et la gestion du foyer, et devient capr

In [47]:
# Résumé des chapitres de la PREMIÈRE PARTIE avec ta fonction
import time
import openai

def get_completion_from_messages(messages, 
                                 model="gpt-4.1", 
                                 temperature=0, 
                                 max_tokens=1000
                                ):
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content



system_message = "Tu es un assistant expert en analyse littéraire française."
delimiter = "```"

def resumer_chapitres_partie1(chapitres_dict):
    resumes = {}

    for numero, contenu in chapitres_dict.items():
        print(f"📖 Résumé du chapitre {numero} en cours...")

        # On crée le prompt pour ce chapitre
        user_message = f"""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{contenu}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui résume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- Environ 100-200 mots

Résumé :
"""

        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": f"{delimiter}{user_message}{delimiter}"}
        ]

        try:
            resume = get_completion_from_messages(messages)
            resumes[numero] = resume
            print(f"✅ Chapitre {numero} résumé.")
        except Exception as e:
            print(f"❌ Erreur pour le chapitre {numero}: {e}")
            resumes[numero] = None
        
        time.sleep(2)  # Petite pause pour éviter une surcharge ou limite d’API

    return resumes

# Exécuter pour la première partie
resumes_partie1 = resumer_chapitres_partie1(chapitres_structures["PREMIÈRE PARTIE"])

# Affichage d’un résumé exemple
print("\n--- Résumé du chapitre I ---\n")
print(resumes_partie1["I"])

📖 Résumé du chapitre I en cours...
✅ Chapitre I résumé.
📖 Résumé du chapitre II en cours...
✅ Chapitre II résumé.
📖 Résumé du chapitre III en cours...
✅ Chapitre III résumé.
📖 Résumé du chapitre IV en cours...
✅ Chapitre IV résumé.
📖 Résumé du chapitre V en cours...
✅ Chapitre V résumé.
📖 Résumé du chapitre VI en cours...
✅ Chapitre VI résumé.
📖 Résumé du chapitre VII en cours...
✅ Chapitre VII résumé.
📖 Résumé du chapitre VIII en cours...
✅ Chapitre VIII résumé.
📖 Résumé du chapitre IX en cours...
❌ Erreur pour le chapitre IX: Error code: 429 - {'error': {'message': 'Request too large for gpt-4.1 in organization org-krUVJ6bzIfce6qGLmjZVrH2m on tokens per min (TPM): Limit 30000, Requested 156438. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

--- Résumé du chapitre I ---

Titre : L’enfance et la jeunesse de Charles Bovary

Ré

In [54]:
for i in resumes_partie1:
    print(f" Chapitre {i} ")
    print(resumes_partie1[i])
    print("\n")

 Chapitre I 
Titre : L’enfance et la jeunesse de Charles Bovary

Résumé :  
Ce premier chapitre de Madame Bovary s’ouvre sur l’arrivée de Charles Bovary, nouvel élève maladroit et timide, dans une classe de collège. Raillé par ses camarades à cause de son apparence provinciale et de sa fameuse casquette, Charles se distingue par sa discrétion et son application. Le récit retrace ensuite son enfance : fils d’un père instable et autoritaire et d’une mère dévouée mais frustrée, Charles grandit à la campagne, choyé par sa mère qui nourrit pour lui de grandes ambitions. Son éducation, d’abord confiée au curé du village, reste lacunaire jusqu’à son entrée au collège de Rouen, où il mène une existence médiocre et sans éclat. Plus tard, étudiant en médecine à Rouen, il peine à s’adapter, échoue à son premier examen, puis réussit grâce à la ténacité de sa mère. Devenu officier de santé, il s’installe à Tostes, où sa mère lui arrange un mariage avec une veuve, Madame Dubuc. Ce chapitre pose les 

In [69]:
chapter_summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 100-200 mots

Résumé :
""",
)

llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)

# Exemple avec un chunk du livre
context = part1_chap9


result = chapter_summary.run(context=context)
print(result)

**Titre : L’ennui d’Emma à Tostes et le rêve d’une autre vie**

Dans ce chapitre, Emma Bovary s’enfonce dans l’ennui et la mélancolie, insatisfaite de sa vie monotone à Tostes auprès de son mari Charles. Elle s’évade dans ses rêveries, nourries par la lecture de romans et de journaux mondains, idéalisant Paris et la haute société. Le souvenir du Vicomte et le porte-cigares en soie verte symbolisent ses désirs inassouvis. Emma tente d’introduire du raffinement dans son quotidien, mais la médiocrité de son environnement et la simplicité de Charles la désespèrent. Elle devient capricieuse, négligente, et son état de santé se détériore. Charles, inquiet, décide de déménager à Yonville pour améliorer sa condition. Le chapitre s’achève sur le départ du couple, Emma étant enceinte, après qu’elle a brûlé son bouquet de mariage, geste symbolique de son désenchantement et de la rupture avec ses illusions passées. Les personnages principaux sont Emma et Charles Bovary, avec quelques figures secon